<img src="http://openenergy-platform.org/static/OEP_logo_2_no_text.svg" alt="OpenEnergy Platform" height="100" width="100"  align="left"/>
<img src="http://reiner-lemoine-institut.de//wp-content/uploads/2015/09/rlilogo.png" alt="RLI" height="100" width="100" align="right"/>

# OpenEnergyPlatform
<br>

## Usage of the BNetzA-MaStR-API (WSDL)
Webservice: https://www.marktstammdatenregister.de/<br>
Dokumentation Zeep: https://python-zeep.readthedocs.io/en/master/<br>

In [1]:
__copyright__ = "© Reiner Lemoine Institut"
__license__   = "GNU Affero General Public License Version 3 (AGPL-3.0)"
__url__       = "https://www.gnu.org/licenses/agpl-3.0.html"
__author__    = "Ludee"

In [2]:
import pandas as pd
import getpass

import zeep
from zeep import Client
from zeep.helpers import serialize_object
from zeep.wsse.username import UsernameToken
from zeep.transports import Transport

from requests import Session
from requests.auth import HTTPBasicAuth  # or HTTPDigestAuth, or OAuth1, etc.

from IPython.core.display import HTML

## Test Zeep

In [3]:
zeep_wsdl = 'http://www.soapclient.com/xml/soapresponder.wsdl'
zeep_client = zeep.Client(wsdl=zeep_wsdl)
print(zeep_client.service.Method1('Zeep', 'is cool'))

Your input parameters are Zeep and is cool


## Reading data from "Test-Webdienst"

Dokumentation: <a href='https://www.marktstammdatenregister.de/MaStRHilfe/files/webdienst/2019-01_31%20WSDL%20MaStR%20Webdienste%20V1.2%20-%20Testsystem.zip' target='_blank'>Dienstbeschreibung 1.2</a><br>
WSDL: https://test.marktstammdatenregister.de/MaStRAPI/wsdl/mastr.wsdl

In [4]:
test_wsdl = 'https://test.marktstammdatenregister.de/MaStRAPI/wsdl/mastr.wsdl'
test_client = zeep.Client(wsdl=test_wsdl)
LokaleUhrzeit = test_client.service.GetLokaleUhrzeit()
print(LokaleUhrzeit)

Error during xml -> python translation
Traceback (most recent call last):
  File "C:\Anaconda3\envs\py36_mastr\lib\site-packages\zeep\xsd\types\simple.py", line 61, in parse_xmlelement
    return self.pythonvalue(xmlelement.text)
  File "C:\Anaconda3\envs\py36_mastr\lib\site-packages\zeep\xsd\types\builtins.py", line 148, in pythonvalue
    return isodate.parse_datetime(value)
  File "C:\Anaconda3\envs\py36_mastr\lib\site-packages\isodate\isodatetime.py", line 56, in parse_datetime
    tmptime = parse_time(timestring)
  File "C:\Anaconda3\envs\py36_mastr\lib\site-packages\isodate\isotime.py", line 133, in parse_time
    tzinfo)
ValueError: second must be in 0..59


{
    'Ergebniscode': 'OK',
    'AufrufVeraltet': False,
    'AufrufLebenszeitEnde': None,
    'AufrufVersion': 1,
    'LokaleUhrzeit': datetime.datetime(2019, 2, 2, 15, 39, 55, 961877, tzinfo=<FixedOffset '+01:00'>)
}


How do you prevent this error?

In [5]:
LocalTime = serialize_object(LokaleUhrzeit)
df = pd.Series(LocalTime).to_frame('value').reset_index()
print(df)

                  index                             value
0          Ergebniscode                                OK
1        AufrufVeraltet                             False
2  AufrufLebenszeitEnde                              None
3         AufrufVersion                                 1
4         LokaleUhrzeit  2019-02-02 15:39:55.961877+01:00


In [6]:
date = df.loc[df['index'] == 'LokaleUhrzeit']
print(date)

           index                             value
4  LokaleUhrzeit  2019-02-02 15:39:55.961877+01:00


## Reading data from "Produktions-Webdienst"

Dokumentation: <a href='https://www.marktstammdatenregister.de/MaStRHilfe/files/webdienst/2019-01_31%20WSDL%20MaStR%20Webdienste%20V1.2%20-%20Produktion.zip' target='_blank'>Dienstbeschreibung 1.2</a><br>
WSDL: https://www.marktstammdatenregister.de/MaStRAPI/wsdl/mastr.wsdl

### Without user (only _GetLokaleUhrzeit_ possible)

https://www.marktstammdatenregister.de/MaStRHilfe/subpages/webdienst.html<br>
**Welche Zugangsberechtigungen sind erforderlich, um den Webdienst zu nutzen?**

_Die Funkion GetLokaleUhrzeit ist die einzige Funktion, für die keine Authentifizierung erforderlich ist. Sie dient im Wesentlichen dazu, die Verbindung zwischen Client und Server zu testen. Für alle anderen Webdienst-Funktionen ist eine Authentifizierung des Benutzers erforderlich. Dies erfolgt durch die Übermittlung eines Webdienst-Key des Webdienst-Benutzers._

In [7]:
wsdl = 'https://www.marktstammdatenregister.de/MaStRAPI/wsdl/mastr.wsdl'
client = zeep.Client(wsdl=wsdl)
LokaleUhrzeit = client.service.GetLokaleUhrzeit()
print(LokaleUhrzeit)

Error during xml -> python translation
Traceback (most recent call last):
  File "C:\Anaconda3\envs\py36_mastr\lib\site-packages\zeep\xsd\types\simple.py", line 61, in parse_xmlelement
    return self.pythonvalue(xmlelement.text)
  File "C:\Anaconda3\envs\py36_mastr\lib\site-packages\zeep\xsd\types\builtins.py", line 148, in pythonvalue
    return isodate.parse_datetime(value)
  File "C:\Anaconda3\envs\py36_mastr\lib\site-packages\isodate\isodatetime.py", line 56, in parse_datetime
    tmptime = parse_time(timestring)
  File "C:\Anaconda3\envs\py36_mastr\lib\site-packages\isodate\isotime.py", line 133, in parse_time
    tzinfo)
ValueError: second must be in 0..59


{
    'Ergebniscode': 'OK',
    'AufrufVeraltet': False,
    'AufrufLebenszeitEnde': None,
    'AufrufVersion': 1,
    'LokaleUhrzeit': datetime.datetime(2019, 2, 2, 15, 40, 8, 592111, tzinfo=<FixedOffset '+01:00'>)
}


In [8]:
LocalTime = serialize_object(LokaleUhrzeit)
df = pd.Series(LocalTime).to_frame('value').reset_index()
print(df)

                  index                             value
0          Ergebniscode                                OK
1        AufrufVeraltet                             False
2  AufrufLebenszeitEnde                              None
3         AufrufVersion                                 1
4         LokaleUhrzeit  2019-02-02 15:40:08.592111+01:00


### Registered MaStR user

You must be registered to use the API. Link to <a href='https://www.marktstammdatenregister.de/MaStR/Assistent/AuswahlAssistent' target='_blank'>register</a>.


In [ ]:
wsdl = 'https://www.marktstammdatenregister.de/MaStRAPI/wsdl/mastr.wsdl'
apiKey = getpass.getpass('Api-Key:')
myMastrNr = getpass.getpass('MaStR-Nr.:')
myMarktrolleNr = getpass.getpass('Marktrolle-Nr.:')
client = Client(wsdl)

In [ ]:
# GetLokaleUhrzeitMitAuthentifizierung
c = client.service.GetLokaleUhrzeitMitAuthentifizierung(apiKey=apiKey)
s = serialize_object(c)
df = pd.Series(s).to_frame('value').reset_index()
print(df)

In [ ]:
# GetListeKatalogkategorien
c = client.service.GetListeKatalogkategorien(apiKey=apiKey)
s = serialize_object(c)
df = pd.Series(s).to_frame('value').reset_index()
print(df)

In [ ]:
# GetAktuellerStandTageskontingent
c = client.service.GetAktuellerStandTageskontingent(apiKey=apiKey, marktakteurMastrNummer=myMarktrolleNr,)
s = serialize_object(c)
df = pd.Series(s).to_frame('value').reset_index()
print(df)

In [ ]:
# GetListeAlleMarktakteure
c = client.service.GetListeAlleMarktakteure(apiKey=apiKey)
s = serialize_object(c)
df = pd.Series(s).to_frame('value').reset_index()
print(df)

In [ ]:
# GetListeFreigegebeneLokationen
c = client.service.GetListeFreigegebeneLokationen(apiKey=apiKey,
                                          marktakteurMastrNummer=myMarktrolleNr)
s = serialize_object(c)
df = pd.Series(s).to_frame('value').reset_index()
print(df)